In [16]:
#Load packages
import pulp
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import time


In [17]:
class Battery():

    def __init__(self, 
                 time_horizon,
                 max_disharge_power_capacity,
                 max_charge_power_capacity):
        #set up decision variables for optimization.
        #Hourly cahrge and discharge flows with limitations
        #Optimization horizon, hours
        self.time_horizon = time_horizon

        self.charge = \
        pulp.LpVariable.dicts(
            "charging power",
            ('c_t_' + str(i) for i in range(0, time_horizon)),
            lowBound=0, upBound=max_charge_power_capacity,
            cat='Continuous')
        
        self.discharge = \
        pulp.LpVariable.dicts(
            "discharging power",
            ('d_t_' + str(i) for i in range(0, time_horizon)),
            lowBound=0, upBound=max_disharge_power_capacity,
            cat='Continuous')
        
    def set_objective(self, prices):
        #create a model and objective funciton.
        #using price data, one price per point on horizon.
        #future implementation; add price likelihoods > Expected Value
        try:
            assert len(prices) == self.time_horizon
        except:
            print('Error: need one price for each hour in time horizon')

            #Instantiate Linear Programming model to maximize objective
            self.model = pulp.LpProblem("Energy arbitrage",
                                        pulp.LpMaximize)
            
            #OBJECTIVE FUNCTION: PROFIT
            #Daily Profit from charging/discharging. 
            #Charging as cost, discharging as revenue.
            self.model += \
            pulp.LpAffineExpression(
                [(self.charge['c_t' + str(i)],
                  -1*prices[i]) for i in range(0, self.time_horizon)]) +\
            pulp.LpAffineExpression(
                [(self.discharge['d_t' + str(i)],
                  prices[i]) for i in range(0, self.time_horizon)])
            
    def storage_contraints(self,
                               efficiency,
                               min_capacity,
                               max_capacity,
                               initial_level):
        #minimum storage level constraint
        #round trip efficiency: energy available for disharge x energy charged
        for hour_of_sim in range(1, self.time_horizon+1):
            self.model += \
            initial_level \
            + pulp.LpAffineExpression(
                [(self.charge['c_t_' + str(i)], efficiency)
                 for i in range(0, hour_of_sim)]) \
            - pulp.lpSum(
                self.discharge[index]
                for index in('d_t_' + str(i)
                             for i in range(0, hour_of_sim)))\
            >= min_capacity

        #Storage Level Constraint 2
        #Max Energy Capacity - Min Capacity = Discharge Energy Capacity
        for hour_of_sim in range(1, self.time_horizon+1):
            self.model += \
            initial_level \
            + pulp.LpAffineExpression(
                [(self.charge['c_t_' + str(i)], efficiency)
                 for i in range(0, hour_of_sim)]) \
            - pulp.lpSum(
                self.discharge[index]
                for index in('d_t_' + str(i)
                             for i in range(0, hour_of_sim)))\
            <= max_capacity

    def throughput_constraints(self,
                                max_daily_discharged_throughput):
        #Max Discharge Throughput Constraint
        #The sum of all discharge flow within a day cannot exceed this
        #Include portion of the next day according to time horizon
        #Assumes the time horizon is at least 24 hours
        #Should be replaced with a discharge cost function, see battery opt research
        self.model += \
        pulp.lpSum(
            self.discharge[index] for index in (
                'd_t_' + str(i) for i in range(0, 24))) \
            <= max_daily_discharged_throughput

        self.model += \
        pulp.lpSum(
            self.dischacharge[index] for index in range(25, self.time_horizon))\
            <= max_daily_discharged_throughput \
            *float(self.time_horizon-24)/24

    def solve_model(self):
        #solve optimization problem, subject to constraints
        self.model.solve()

        #show warning if optimal solution not found
        if pulp.LpStatus[self.model.status] != 'Optimal':
            print('Warning:' + pulp.LpStatus[self.model.status])

    def collect_output(self):
        #collect charging and discharging rates within time horizon
        hourly_charges =\
            np.array(
                [self.cahrge[index].varValue for 
                index in ('c_t_' + str(i) for i in range(0, 24))])
        hourly_discharges =\
            np.array(
                [self.discharge[index].varValue for
                index in ('d_t_' + str(i) for i in range(0, 24))])
        return hourly_charges, hourly_discharges

In [18]:
#import data from directory
data_directory = 'data_2019_2020_from_web/'

dir_list = os.listdir(data_directory)
dir_list.sort()

for item in dir_list: #Remove invisible files (i.e. .DS_Store used by Mac OS)
    if item[0] == '.':
        dir_list.remove(item)

dir_list

tic = time.time()
#count loaded files
file_counter = 0

#Load csvs into a DataFrame
for this_sub_dir in dir_list:
    #List the files
    this_sub_dir_list = os.listdir(data_directory + '/' + this_sub_dir)
    #Sort the list
    this_sub_dir_list.sort()
    #Delete invisible files (that start with '.')
    for this_item in this_sub_dir_list:
        if this_item[0] == '.':
            this_sub_dir_list.remove(this_item)
    #For each file in the subdirectory
    for this_file in this_sub_dir_list:
        #Load the contents into a DataFrame
        this_df = pd.read_csv(data_directory + '/' + this_sub_dir + '/' + this_file)
        #Concatenate with existing data if past first file
        if file_counter == 0:
            all_data = this_df.copy()
        else:
            all_data = pd.concat([all_data, this_df])
        
        file_counter += 1
toc = time.time()
print(str(toc-tic) + ' seconds to load ' + str(file_counter) + ' files')

# all_data.info()
all_data.head()

2.656618356704712 seconds to load 366 files


,Time Stamp,Name,PTID,LBMP ($/MWHr),Marginal Cost Losses ($/MWHr),Marginal Cost Congestion ($/MWHr)
0,05/01/2019 00:00,CAPITL,61757,20.43,0.93,-4.04
1,05/01/2019 00:00,CENTRL,61754,16.17,0.15,-0.55
2,05/01/2019 00:00,DUNWOD,61760,20.13,1.50,-3.17
3,05/01/2019 00:00,GENESE,61753,15.62,-0.26,-0.43
4,05/01/2019 00:00,H Q,61844,15.09,-0.37,0.00


In [19]:
# Data Info
unique_names = all_data['Name'].unique()
print(len(unique_names))
unique_names

# check shape matches expected time horizons (time zones * hours * days)
assert 15*24*366 == all_data.shape[0]

# Filter data to only include the zone of interest (NYC)
zone_of_interest = 'N.Y.C.'
all_data = all_data.loc[all_data['Name'].isin([zone_of_interest]),:]
all_data.shape

# Set datetime indexing
all_data = all_data.set_index(['Time Stamp'])
all_data.index = pd.to_datetime(all_data.index, format='%m/%d/%Y %H:%M')

15


In [20]:
# batter simulation

def simulate_battery(initial_level,
                     price_data,
                     max_discharge_power_capacity,
                     max_charge_power_capacity,
                     discharge_energy_capacity,
                     efficiency,
                     max_daily_discharged_throughput,
                     time_horizon,
                     start_day):
    #Track simulation time
    tic = time.time()
    
    #Initialize output variables
    all_hourly_charges = np.empty(0)
    all_hourly_discharges = np.empty(0)
    all_hourly_state_of_energy = np.empty(0)
    all_daily_discharge_throughput = np.empty(0)
    
    #Set up decision variables for optimization by
    #instantiating the Battery class
    battery = Battery(
        time_horizon=time_horizon,
        max_discharge_power_capacity=max_discharge_power_capacity,
        max_charge_power_capacity=max_charge_power_capacity)
    
    #############################################
    #Run the optimization for each day of the year.
    #############################################
    
    #There are 365 24-hour periods (noon to noon) in the simulation,
    #contained within 366 days
    for day_count in range(365):
        #print('Trying day {}'.format(day_count))
        
        #############################################
        ### Select data and simulate daily operation
        #############################################
        
        #Set up the 36 hour optimization horizon for this day by
        #adding to the first day/time of the simulation
        start_time = start_day \
        + pd.Timedelta(day_count, unit='days')
        end_time = start_time + pd.Timedelta(time_horizon-1, unit='hours')
        #print(start_time, end_time)
    
        #Retrieve the price data that will be used to calculate the
        #objective
        prices = \
        price_data[start_time:end_time]['LBMP ($/MWHr)'].values
                      
        #Create model and objective
        battery.set_objective(prices)

        #Set storage constraints
        battery.add_storage_constraints(
            efficiency=efficiency,
            depth_of_discharge=depth_of_discharge,
            battery_capacity=battery_capacity,
            initial_level=initial_level)
            
        #Set maximum discharge throughput constraint
        battery.add_throughput_constraints(
            max_daily_discharged_throughput=
            max_daily_discharged_throughput)

        #Solve the optimization problem and collect output
        battery.solve_model()
        hourly_charges, hourly_discharges = battery.collect_output()
        
        #############################################
        ### Manipulate daily output for data analysis
        #############################################
        
        #Collect daily discharge throughput
        daily_discharge_throughput = sum(hourly_discharges)
        #Calculate net hourly power flow (kW), needed for state of energy.
        #Charging needs to factor in efficiency, as not all charged power
        #is available for discharge.
        net_hourly_activity = (hourly_charges*efficiency) \
        - hourly_discharges
        #Cumulative changes in energy over time (kWh) from some baseline
        cumulative_hourly_activity = np.cumsum(net_hourly_activity)
        #Add the baseline for hourly state of energy during the next
        #time step (t2)
        state_of_energy_from_t2 = initial_level \
        + cumulative_hourly_activity
        
        #Append output
        all_hourly_charges = np.append(all_hourly_charges, hourly_charges)
        all_hourly_discharges = np.append(
            all_hourly_discharges, hourly_discharges)
        all_hourly_state_of_energy = \
        np.append(all_hourly_state_of_energy, state_of_energy_from_t2)
        all_daily_discharge_throughput = \
        np.append(
            all_daily_discharge_throughput, daily_discharge_throughput)
        
        #############################################
        ### Set up the next day
        #############################################
        
        #Initial level for next period is the end point of current period
        initial_level = state_of_energy_from_t2[-1]
        
        

    toc = time.time()
        
    print('Total simulation time: ' + str(toc-tic) + ' seconds')

    return all_hourly_charges, all_hourly_discharges, \
        all_hourly_state_of_energy,\
        all_daily_discharge_throughput

In [21]:
# Parameter Setup 
battery_capacity = 80 #(kWh)
depth_of_discharge = 0.01 #unitless
discharge_energy_capacity = 100 #(kWh)
max_discharge_power_capacity = 80 #(kW)
max_charge_power_capacity = 80 #(kW)
efficiency = 0.91 #unitless
max_daily_discharged_throughput = 400  #(kWh)
initial_level = 0.75*battery_capacity #kWh (75% of capacity)

#Run the simulation
all_hourly_charges, all_hourly_discharges, all_hourly_state_of_energy,\
all_daily_discharge_throughput = \
simulate_battery(initial_level=initial_level,
                 price_data=all_data,
                 max_discharge_power_capacity
                     =max_discharge_power_capacity,
                 max_charge_power_capacity
                     =max_charge_power_capacity,
                 discharge_energy_capacity=discharge_energy_capacity,
                 efficiency=efficiency,
                 max_daily_discharged_throughput
                     =max_daily_discharged_throughput,
                 time_horizon=36,
                 start_day=pd.Timestamp(
                     year=2019, month=5, day=1, hour=12,
                     tz='America/New_York'))
assert 24*365 == len(all_hourly_discharges)

TypeError: Battery.__init__() got an unexpected keyword argument 'max_discharge_power_capacity'

In [ ]:
mpl.rcParams["figure.figsize"] = [5,3]
mpl.rcParams["figure.dpi"] = 100
mpl.rcParams.update({"font.size":12})

plt.hist(all_hourly_discharges - all_hourly_charges)
plt.xlabel('kW')
plt.title('Hourly power output vs Num of Occurrences/yr')

NameError: name 'all_hourly_discharges' is not defined